## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,72.55,90,48,7.00,scattered clouds
1,1,Saint George,US,37.1041,-113.5841,96.64,31,1,4.00,clear sky
2,2,Coahuayana,MX,18.7333,-103.6833,90.88,55,6,5.35,clear sky
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,68,20,13.80,few clouds
4,4,Jamestown,US,42.0970,-79.2353,70.30,75,75,4.00,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 125


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,72.55,90,48,7.00,scattered clouds
1,1,Saint George,US,37.1041,-113.5841,96.64,31,1,4.00,clear sky
2,2,Coahuayana,MX,18.7333,-103.6833,90.88,55,6,5.35,clear sky
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,67.86,68,20,13.80,few clouds
4,4,Jamestown,US,42.0970,-79.2353,70.30,75,75,4.00,broken clouds
5,5,Salalah,OM,17.0151,54.0924,77.09,94,90,4.61,overcast clouds
6,6,Nikolskoye,RU,59.7035,30.7861,64.02,78,5,6.33,clear sky
7,7,Saint-Pierre,RE,-21.3393,55.4781,67.64,72,20,6.91,few clouds
8,8,Ushuaia,AR,-54.8000,-68.3000,46.06,49,20,17.27,few clouds
9,9,Ponta Do Sol,PT,32.6667,-17.1000,73.71,76,1,5.41,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        643
City           643
Country        637
Lat            643
Lng            643
Max Temp       643
Humidity       643
Cloudiness     643
Wind Speed     643
Description    643
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID        637
City           637
Country        637
Lat            637
Lng            637
Max Temp       637
Humidity       637
Cloudiness     637
Wind Speed     637
Description    637
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,72.55,scattered clouds,-0.7393,-90.3518,
1,Saint George,US,96.64,clear sky,37.1041,-113.5841,
2,Coahuayana,MX,90.88,clear sky,18.7333,-103.6833,
3,Arraial Do Cabo,BR,67.86,few clouds,-22.9661,-42.0278,
4,Jamestown,US,70.30,broken clouds,42.0970,-79.2353,
5,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
6,Nikolskoye,RU,64.02,clear sky,59.7035,30.7861,
7,Saint-Pierre,RE,67.64,few clouds,-21.3393,55.4781,
8,Ushuaia,AR,46.06,few clouds,-54.8000,-68.3000,
9,Ponta Do Sol,PT,73.71,clear sky,32.6667,-17.1000,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [16]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

637
570


In [17]:
clean_hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [18]:
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,72.55,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,Saint George,US,96.64,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
2,Coahuayana,MX,90.88,clear sky,18.7333,-103.6833,El rranchito michuacan de ocampos
3,Arraial Do Cabo,BR,67.86,few clouds,-22.9661,-42.0278,Pousada Porto Praia
4,Jamestown,US,70.30,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
675,Imeni Poliny Osipenko,RU,50.41,overcast clouds,52.4201,136.4853,Yubileynaya
676,Nicoya,CR,89.15,overcast clouds,10.1483,-85.4520,Guayacan Real
677,Port Blair,IN,81.81,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
678,Dzilam Gonzalez,MX,84.85,broken clouds,21.2833,-88.9333,posadas


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
#Let's add the markers for each city on top of the heatmap.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))